## Лаба 4. Прогнозирование пола и возрастной категории — Spark Streaming

#### Дедлайн

Понедельник, 22 марта, 23:59.

#### Дедлайн Github

Четверг, 25 марта, 23:59

#### Задача

У вас имеются данные: логи посещения пользователей по разным сайтам рунета. По этим пользователям вам также известны: пол и возрастная категория. Вам нужно будет в real-time спрогнозировать эти характеристики по пользователям, о которых у вас нет этой информации, но будут все те же самые логи посещения.

##### При решении задачи запрещено использовать библиотеки pandas, sklearn (кроме sklearn.metrics), xgboost и прочие.

#### Описание данных

Обучающая выборка, с которой вы будете работать, выглядит следующим образом:

```json
gender	age	uid	user_json
F	18-24	d50192e5-c44e-4ae8-ae7a-7cfe67c8b777	{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}
```

Она расположена на hdfs: `/labs/slaba04/`.

Поле `gender` принимает значения `F` (женщина) и `M` (мужчина).

Поле `age` принимает значения диапазона возраста: `18-24`, `25-34`, `35-44`, `45-54`, `>=55`

Поле `uid` принимает значения уникального ID пользователя (cookies): `d50192e5-c44e-4ae8-ae7a-7cfe67c8b777`.

Поле `user_json` имеет внутри json со следующей схемой данных: `{"visits": [{"url": "url1", "timestamp": "timestamp1"}, {"url": "url2", "timestamp": "timestamp2"}]}`. В нем содержатся непосредственно логи посещения пользователем страниц вместе с временной меткой посещения.

Данные, по которым вам надо будет построить прогноз (тестовая выборка), хранятся в Kafka в таком виде:

```json
{"uid": "bdd48781-8243-493d-8ae6-794050a4417f",
 "visits": "[{"url": "http://vk.com/feed", "timestamp": 1425077901001}, {"url": "http://big-cards.a5ltd.com/app/html/vk.html?api_url=http://api.vk.com/api.php&api_id=1804162&api_settings=663823&viewer_id=40849488&viewer_type=2&sid=1b77e9ca975573fed6e31746ca58ea2509f3588d918e7dab2a291f7f7579ae7dab6c21058c837b74b1fbb&secret=37339dfcbf&access_token=06c53bd8f9f9a0afad23b4e166e448495396259fc5a1c5e5243474df82591bd5d0afe877d993ea4a82a81&user_id=40849488&group_id=0&is_app_user=1&auth_key=d7c91ad4bb27e20cb3ed81810f5649db&language=0&parent_language=0&ad_info=elsdcqvcsvftaqxtawjsxht b0q8htjxuvbbjrvbnwojfji2ha8h&is_secure=0&ads_app_id=1804162_903aa103fa48b1e378&referrer=menu&lc_name=2c073eed&hash=", "timestamp": 1425077901000}]"}
```

То есть все то же самое, только без поля `gender` и `age` и немного с другой схемой данных. Название топика, в который мы будем присылать данные: `input_ivan.ivanov`, где вместо `ivan.ivanov` ваш логин от личного кабинета. Данные вам будут поступать по нажатию кнопки "Проверить" в личном кабинете. Вам топик создавать не нужно. Он будет автоматом создаваться при записи. Чистить его тоже не надо, потому что Spark Streaming хранит оффсеты, то есть знает, что он уже считывал, что еще не считывал. 

Порт брокера Kafka: `6667`. Hostname: `spark-master-1.newprolab.com` или `spark-node-1.newprolab.com`.

#### Результат

Вам в свою очередь нужно будет, считывая данные из Kafka, делать прогноз по ним в Spark Streaming и отправлять в real-time эти прогнозные значения в свой другой топик в Kafka в формате:

```json
{"uid": "fe1dba8f-3131-439f-9031-851c0da0f126", "gender": "M", "age": "25-34"}
{"uid": "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777", "gender": "F", "age": "18-24"}
```

Название топика — ваш логин в личном кабинете: `ivan.ivanov`. После отправки данных чекер будет ждать 90 секунд, чтобы вы обработали входные данные из топика `input_ivan.ivanov` и записали результаты прогноза уже в этот топик. 

#### Подсказки

Чтение из Kafka:

```python
read_kafka_params = {
    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
    "subscribe": "input_ivan.ivanov",
    "startingOffsets": "latest"
}
kafka_sdf = spark.readStream.format("kafka").options(**read_kafka_params).load()
```

Запись в Kafka:

```python
write_kafka_params = {
   "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
   "topic": "ivan.ivanov"
}
batch_df.writeStream.format("kafka").options(**write_kafka_params)\
    .option("checkpointLocation", "streaming/chk/chk_kafka")\
    .outputMode("append").start()
```
  
Бывают проблемы при чтении старых данных, которых уже нет в кафке:
  
```
.option("failOnDataLoss", 'False')
```
  
#### Проверка
Проверка лабы устроена следующим образом:
1. Вы готовите свою модель
2. Настраиваете чтение данных из одного топика и запись в другой топик.
4. Запускаете свой скрипт в стриминговым режиме
3. Идет подключение к топику на чтение данных.
4. На странице лабы нажимаете кнопку "Проверить".
5. Вам в топик текут данные.
6. Модель обрабатывает входные данные.
7. Ваш скрипт сохраняет данные в другой топик.
8. Чекер на странице лабы выдает результат проверки.


Проверка осуществляется автоматическим скриптом на странице лабы в личном кабинете. Наш чекер в конечном итоге подключится к вашему финальному топику в Kafka и сравнит ваш прогноз с эталонным ответом. На выходе вы получите свой score. В качестве него метрика `accuracy`: доля пользователей от общего числа, по которым вы верно угадали и пол, и возрастную категорию. Если `accuracy` будет больше **0.25**, то лаба будет засчитана.

##### Особенности проверки
1. Проверка (генерация событий, обработка вашим скриптом, чтение и проверка результатов) занимают до 90 секунд, просьба ожидать (можно нажимать кнопку "обновить")
2. За один раз (одно нажание "Проверить") чекер присылает батч в 5000 событий. Если для отладки / повторной проверки требуется еще, надо нажимать снова

####  Рекомендации по прохождению
1. Разработанную модель (и вообще весь пайплайн) можно сохранить в HDFS и потом загрузить уже в стриминговом скрипте (методы save/load - см.документацию)
2. Настоятельно рекомендуется сначала отладить весь процесс чтения данных, предсказания и сохранения результатов в **батчевом** режиме (сгенерить порцию событий чекером и потом считать ее в обычный DataFrame через  `spark.read.format("kafka")`. И уже только после проверки полной  работоспособсности всего скрипта и корректности формата выходных данных в Кафке, переписать на стриминговый режим (благодаря единому API Spark, это не потребует больших изменений)

Обязательное условие зачета лабораторной работы – это выкладка после дедлайна лабы своего решения в репозиторий через pull-request. Как это сделать, можно прочитать [здесь](/git.md). Если будут вопросы – спрашивайте в Slack.


In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("lab04_RysinN").getOrCreate()

In [4]:
spark

In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, StringIndexer, HashingTF, RegexTokenizer, StopWordsRemover, IDF, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark import keyword_only
from pyspark.ml.param import Param, Params, TypeConverters
from pyspark.ml import Transformer

from pyspark.ml.classification import LogisticRegression, GBTClassifier

In [6]:
import re
import json

In [7]:
path_to_train_data = '/labs/slaba04/'

In [8]:
!hdfs dfs -ls /labs/slaba04/

Found 1 items
-rw-r--r--   3 hdfs hdfs  655090069 2021-02-27 22:13 /labs/slaba04/gender_age_dataset.txt


In [9]:
!hadoop fs -du -s /labs/slaba04/  | awk '{s+=$1} END {printf "%.3fGB\n", s/1000000000}'

0.655GB


## dev model

In [10]:
schema = StructType([StructField('gender', StringType()), 
                     StructField('age', StringType()),
                     StructField('uid', StringType()),
                     StructField('user_json', StringType())
                    ]
                   )

train_data = spark.read.format("csv")\
                       .option("inferSchema", "true")\
                       .schema(schema)\
                       .option("header", "true")\
                       .option("delimiter", "\\t")\
                       .load(path_to_train_data)

train_data = train_data.filter('age != "-" and gender != "-"')

visits_schema = StructType([
    StructField("visits", ArrayType(
      StructType([
          StructField("url", StringType()),
          StructField("timestamp", LongType())
      ])
   ))
]) 

train_data = train_data.withColumn('visits', 
                                   F.from_json(F.col('user_json'),
                                               schema=visits_schema
                                              )
                                  )

train_data = train_data.withColumn('visited_pages', F.col('visits.visits.url'))

train_data = train_data.withColumn('is_M', F.when(F.col("gender") == 'M', 1).otherwise(0))

train_data = train_data.withColumn("age_category", F.when(F.col("age") == '18-24', 0)\
                                                    .when(F.col("age") == '25-34', 1)\
                                                    .when(F.col("age") == '35-44', 2)\
                                                    .when(F.col("age") == '45-54', 3)\
                                                    .otherwise(4)
                                  )

In [11]:
#train_data.groupby('age_gender').count().show()

In [12]:
train_data.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)
 |-- visits: struct (nullable = true)
 |    |-- visits: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- url: string (nullable = true)
 |    |    |    |-- timestamp: long (nullable = true)
 |-- visited_pages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_M: integer (nullable = false)
 |-- age_category: integer (nullable = false)



In [13]:
train_data.show(5)

+------+-----+--------------------+--------------------+--------------------+--------------------+----+------------+
|gender|  age|                 uid|           user_json|              visits|       visited_pages|is_M|age_category|
+------+-----+--------------------+--------------------+--------------------+--------------------+----+------------+
|     F|18-24|d50192e5-c44e-4ae...|{"visits": [{"url...|[[[http://zebra-z...|[http://zebra-zoy...|   0|           0|
|     M|25-34|d502331d-621e-472...|{"visits": [{"url...|[[[http://sweetra...|[http://sweetradi...|   1|           1|
|     F|25-34|d50237ea-747e-48a...|{"visits": [{"url...|[[[http://ru.orif...|[http://ru.orifla...|   0|           1|
|     F|25-34|d502f29f-d57a-46b...|{"visits": [{"url...|[[[http://transla...|[http://translate...|   0|           1|
|     M| >=55|d503c3b2-a0c2-4f4...|{"visits": [{"url...|[[[https://mail.r...|[https://mail.ram...|   1|           4|
+------+-----+--------------------+--------------------+--------

In [14]:
@F.udf(returnType=ArrayType(StringType()))
def extract_key_words(user_sites):
    sites_cleaned = [re.sub(r'(http://|https://|www)', '', site) for site in user_sites]    
    sites_cleaned = sum([re.findall(r'\w+', site) for site in sites_cleaned], [])
    sites_cleaned = [site for site in sites_cleaned if site.isalpha()]
    return sites_cleaned

class CleanSitesTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(CleanSitesTransformer, self).__init__()
        if inputCol is not None:
            self.setInputCol(inputCol)
        if outputCol is not None:
            self.setOutputCol(outputCol)
            
    def _transform(self, dataset):
        return dataset.withColumn(self.getOutputCol(), extract_key_words(F.col(self.getInputCol())))

In [15]:
transformer_cleaner = CleanSitesTransformer(inputCol="visited_pages", outputCol="sites_words")

en_stopwords = StopWordsRemover.loadDefaultStopWords("english")
remover = StopWordsRemover(inputCol="sites_words",
                           outputCol="sites_words_filtered",
                           stopWords=en_stopwords)

tf = HashingTF(inputCol="sites_words_filtered", outputCol="tf", numFeatures=15000)

idf = IDF(inputCol="tf", outputCol="tf_idf")

scaler = StandardScaler()\
         .setInputCol("tf_idf")\
         .setOutputCol("tf_idf_norm")


gender_model = LogisticRegression(featuresCol='tf_idf_norm', 
                                  rawPredictionCol='rawPrediction_gender', 
                                  predictionCol='prediction_gender', 
                                  labelCol='is_M', 
                                  maxIter=25)


pipeline1 = Pipeline(stages=[transformer_cleaner, remover, tf, idf, scaler, gender_model])

In [16]:
age_model = LogisticRegression(featuresCol='tf_idf_norm', 
                               rawPredictionCol='rawPrediction_age', 
                               predictionCol='prediction_age', 
                               labelCol='age_category', 
                               maxIter=25, 
                               regParam=0.3)

pipeline2 = Pipeline(stages=[age_model])

In [17]:
pipeline1 = pipeline1.fit(train_data)
scored_train_data = pipeline1.transform(train_data)
scored_train_data = scored_train_data.select([col for col in scored_train_data.columns if col != "probability"])
pipeline2 = pipeline2.fit(scored_train_data)
scored_train_data = pipeline2.transform(scored_train_data)

In [18]:
scored_train_data.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)
 |-- visits: struct (nullable = true)
 |    |-- visits: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- url: string (nullable = true)
 |    |    |    |-- timestamp: long (nullable = true)
 |-- visited_pages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_M: integer (nullable = false)
 |-- age_category: integer (nullable = false)
 |-- sites_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- sites_words_filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- tf_idf_norm: vector (nullable = true)
 |-- rawPrediction_gender: vector (nullable = true)
 |-- prediction_gender: double (nullable = false)
 |-- rawPrediction_age: vec

In [19]:
scored_train_data.show(5, vertical=True)

-RECORD 0------------------------------------
 gender               | F                    
 age                  | 18-24                
 uid                  | d50192e5-c44e-4ae... 
 user_json            | {"visits": [{"url... 
 visits               | [[[http://zebra-z... 
 visited_pages        | [http://zebra-zoy... 
 is_M                 | 0                    
 age_category         | 0                    
 sites_words          | [zebra, zoya, ru,... 
 sites_words_filtered | [zebra, zoya, ru,... 
 tf                   | (15000,[205,1297,... 
 tf_idf               | (15000,[205,1297,... 
 tf_idf_norm          | (15000,[205,1297,... 
 rawPrediction_gender | [4.37902049421712... 
 prediction_gender    | 0.0                  
 rawPrediction_age    | [0.02435235399395... 
 probability          | [0.16114914011656... 
 prediction_age       | 1.0                  
-RECORD 1------------------------------------
 gender               | M                    
 age                  | 25-34     

## batch scoring

In [20]:
read_kafka_params = {
    "kafka.bootstrap.servers": "spark-de-node-1.newprolab.com:6667",
    "subscribe": "input_nikita.rysin",
    "startingOffsets": "earliest"
}

write_kafka_params = {
   "kafka.bootstrap.servers": "spark-de-node-1.newprolab.com:6667",
   "topic": "nikita.rysin"
}

In [21]:
kafka_sdf = spark.read.format("kafka").options(**read_kafka_params).load()

In [22]:
kafka_sdf.show()

+----+--------------------+------------------+---------+------+--------------------+-------------+
| key|               value|             topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------------------+---------+------+--------------------+-------------+
|null|[7B 22 75 69 64 2...|input_nikita.rysin|        0| 60000|2021-03-24 17:41:...|            0|
|null|[7B 22 75 69 64 2...|input_nikita.rysin|        0| 60001|2021-03-24 17:41:...|            0|
|null|[7B 22 75 69 64 2...|input_nikita.rysin|        0| 60002|2021-03-24 17:41:...|            0|
|null|[7B 22 75 69 64 2...|input_nikita.rysin|        0| 60003|2021-03-24 17:41:...|            0|
|null|[7B 22 75 69 64 2...|input_nikita.rysin|        0| 60004|2021-03-24 17:41:...|            0|
|null|[7B 22 75 69 64 2...|input_nikita.rysin|        0| 60005|2021-03-24 17:41:...|            0|
|null|[7B 22 75 69 64 2...|input_nikita.rysin|        0| 60006|2021-03-24 17:41:...|            0|
|null|[7B 

In [23]:
scoring_data_schema = StructType([
    StructField('uid', StringType()),
    StructField('visits', StringType())
]
)

scoring_data_visits_schema = ArrayType(
      StructType([
          StructField("url", StringType()),
          StructField("timestamp", LongType())
      ])
   )

def scoring_batch(batch_df):
    scoring_data = batch_df.withColumn('weblog', F.from_json(F.col('value').cast(StringType()), 
                                                          scoring_data_schema)
                                   ) \
                        .select('weblog.*') \
                        .select('uid', F.from_json(F.col('visits'), 
                                                   scoring_data_visits_schema).alias('visits')
                               )
    
    scoring_data = scoring_data.withColumn('visited_pages', F.col('visits.url'))
    
    scoring_data = pipeline1.transform(scoring_data)
    
    scoring_data = scoring_data.select([col for col in scoring_data.columns if col != "probability"])
    
    scoring_data = pipeline2.transform(scoring_data)
    
    scoring_data = scoring_data.withColumn('gender', F.when(F.col("prediction_gender") == 1, 'M').otherwise('F'))

    scoring_data = scoring_data.withColumn("age", F.when(F.col("prediction_age") == 0, '18-24')\
                                                   .when(F.col("prediction_age") == 1, '25-34')\
                                                   .when(F.col("prediction_age") == 2, '35-44')\
                                                   .when(F.col("prediction_age") == 3, '45-54')\
                                                   .otherwise('>=55')
                                  )
    
    scoring_data = scoring_data.select('uid', 'gender', 'age')
    
    scoring_data = scoring_data.select(F.to_json(F.struct([scoring_data[x] for x in scoring_data.columns])) \
                               .cast(StringType()).alias("value")
                                      )
    return scoring_data

In [24]:
scoring_batch(kafka_sdf).show(5, vertical=True, truncate=False)

(0 rows)



In [ ]:
scoring_batch(kafka_sdf).write.format("kafka").options(**write_kafka_params)\
    .option("checkpointLocation", "streaming/chk/chk_kafka")\
    .save()

In [ ]:
spark.read.format("kafka").options(**{
    "kafka.bootstrap.servers": "spark-de-node-1.newprolab.com:6667",
    "subscribe": "nikita.rysin",
    "startingOffsets": "earliest"
}).load().select(F.col('value').cast(StringType()).alias('row')).show(5, vertical=True, truncate=False)

## streaming scoring

In [25]:
read_kafka_params = {
    "kafka.bootstrap.servers": "spark-de-node-1.newprolab.com:6667",
    "subscribe": "input_nikita.rysin",
    "startingOffsets": "earliest"
}

write_kafka_params = {
   "kafka.bootstrap.servers": "spark-de-node-1.newprolab.com:6667",
   "topic": "nikita.rysin"
}

In [26]:
scoring_data_schema = StructType([
    StructField('uid', StringType()),
    StructField('visits', StringType())
]
)

scoring_data_visits_schema = ArrayType(
      StructType([
          StructField("url", StringType()),
          StructField("timestamp", LongType())
      ])
   )

def scoring_batch(batch_df, epoch_id):
    scoring_data = batch_df.withColumn('weblog', F.from_json(F.col('value').cast(StringType()), 
                                                          scoring_data_schema)
                                   ) \
                        .select('weblog.*') \
                        .select('uid', F.from_json(F.col('visits'), 
                                                   scoring_data_visits_schema).alias('visits')
                               )
    
    scoring_data = scoring_data.withColumn('visited_pages', F.col('visits.url'))
    
    scoring_data = pipeline1.transform(scoring_data)
    
    scoring_data = scoring_data.select([col for col in scoring_data.columns if col != "probability"])
    
    scoring_data = pipeline2.transform(scoring_data)
    
    scoring_data = scoring_data.withColumn('gender', F.when(F.col("prediction_gender") == 1, 'M').otherwise('F'))

    scoring_data = scoring_data.withColumn("age", F.when(F.col("prediction_age") == 0, '18-24')\
                                                   .when(F.col("prediction_age") == 1, '25-34')\
                                                   .when(F.col("prediction_age") == 2, '35-44')\
                                                   .when(F.col("prediction_age") == 3, '45-54')\
                                                   .otherwise('>=55')
                                  )
    
    scoring_data = scoring_data.select('uid', 'gender', 'age')
    
    scoring_data = scoring_data.select(
        F.to_json(
            F.struct([scoring_data[x] for x in scoring_data.columns])
        ).alias("value") #.cast(StringType())
                                      )
    scoring_data.write\
     .format('kafka')\
     .options(**write_kafka_params)\
     .mode('append')\
     .save()
    
    pass

In [27]:
def create_console_sink(df):
    return df.writeStream\
            .foreachBatch(scoring_batch)\
            .option('checkpointLocation', 'streaming/chk/chk_kafka')

In [28]:
kafka_df = spark.readStream.format("kafka").options(**read_kafka_params).option("failOnDataLoss", "False").load()
#kafka_df = kafka_df.selectExpr("CAST(value AS STRING)")

In [29]:
kafka_df.isStreaming

True

In [30]:
sink = create_console_sink(kafka_df)

In [31]:
sq = sink.start()

In [32]:
sq.isActive

True

In [33]:
print(sq.status)

{'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


In [34]:
# .select(F.col('value').cast(StringType()).alias('row'))
spark.read.format("kafka").options(**{
    "kafka.bootstrap.servers": "spark-de-node-1.newprolab.com:6667",
    "subscribe": "nikita.rysin",
    "startingOffsets": "earliest"
}).load().select(F.col('value').cast(StringType()).alias('row'), 'timestamp').show(5, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------
 row       | {"uid":"bd7a30e1-a25d-4cbf-a03f-61748cbe540e","gender":"M","age":"35-44"} 
 timestamp | 2021-03-24 18:23:37.502                                                   
-RECORD 1------------------------------------------------------------------------------
 row       | {"uid":"bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd","gender":"F","age":"25-34"} 
 timestamp | 2021-03-24 18:23:37.691                                                   
-RECORD 2------------------------------------------------------------------------------
 row       | {"uid":"bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c","gender":"M","age":"25-34"} 
 timestamp | 2021-03-24 18:23:37.708                                                   
-RECORD 3------------------------------------------------------------------------------
 row       | {"uid":"bd7c5d7a-0def-41d1-895f-fdb96c56c2d4","gender":"M","age":"35-44"} 
 timestamp | 2021-03-24 18:23:37

## spark.stop()

In [35]:
spark.stop()